# IMAGE PROCESSING #

1. Import Texas School geo data
2. Create 30cm TIF file for each school in Austin ISD
3. ZERO SHOT BUILDING DETECTION (completed in another Studio)
4. Get TIF for each bounding box of the buildings detected

In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import geopandas as gpd
from tqdm.auto import tqdm
import leafmap

Filter Texas School Data - currenlty focused on Austin ISD

In [3]:
schools_geo = gpd.read_file('school_geocodes.geojson')
schools = pd.read_csv('tx_schools.csv', index_col=0)
schools = pd.concat([schools_geo, schools], axis=1)


In [4]:
gdf_aisd = schools[schools['School District'] == 'Austin ISD'].copy().reset_index(drop=True)
# gdf_todo = gdf_aisd.iloc[gdf_aisd[gdf_aisd['School Name'] == 'Bryker Woods Elem'].index[0]:]

Generate TIF files for each school

In [ ]:
bbox_scaler = np.array([-0.00326218, -0.0018414 ,  0.00233782,  0.0020586 ])
for i, school in gdf_aisd.iterrows():
    tif_filename = f'{school["School Name"]}.tif'
    center = school.geometry.centroid.coords[0]

    bbox = (np.array(bbox_scaler) + np.repeat(np.array(center).reshape(1,2), 2, axis=0).flatten()).tolist()
    leafmap.map_tiles_to_geotiff(
        output=os.path.join('data', 'images', tif_filename), bbox=bbox, resolution=.03, source="Satellite", overwrite=True
    )

"""
Visualize the location
"""
# m = leafmap.Map(center=center[::-1], zoom=18, height="800px")
# m.add_basemap('SATELLITE')

Generate TIF files for each building located in each school (this is achieved using geosam in another studio)

In [3]:
geojsons = [f for f in os.listdir('../geosam/data') if f.endswith('.geojson')]

In [11]:
def tif_to_png(file_path, tif_file):
  tiff_image = Image.open(os.path.join(file_path, 'tif', tif_file))

  # Convert the image to JPEG format
  jpeg_image = tiff_image.convert('RGB')

  # Save the JPEG image
  jpeg_image.save(os.path.join(file_path, 'png', tif_file.replace('.tif', '.png')))

In [12]:
for geojson in tqdm(geojsons):
    if not 'Akins' in geojson:
        gdf = gpd.GeoDataFrame.from_file(os.path.join('../geosam/data', geojson))
        school_name = geojson.split('_')[0]
        school_path = os.path.join('data', 'images', school_name)
        os.makedirs(os.path.join(school_path, 'tif'), exist_ok=True)
        os.makedirs(os.path.join(school_path, 'png'), exist_ok=True)
        for i, bounds in tqdm(gdf.bounds.iterrows()):
            tif_filename = f'{school_name} Building {i}.tif'
            bbox = bounds.values.tolist()

            leafmap.map_tiles_to_geotiff(
                output=os.path.join(school_path, 'tif', tif_filename), bbox=bbox, resolution=.03, source="Satellite", overwrite=True
            )
            tif_to_png(school_path, tif_filename)

  0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Downloaded image 1/24
Downloaded image 2/24
Downloaded image 3/24
Downloaded image 4/24
Downloaded image 5/24
Downloaded image 6/24
Downloaded image 7/24
Downloaded image 8/24
Downloaded image 9/24
Downloaded image 10/24
Downloaded image 11/24
Downloaded image 12/24
Downloaded image 13/24
Downloaded image 14/24
Downloaded image 15/24
Downloaded image 16/24
Downloaded image 17/24
Downloaded image 18/24
Downloaded image 19/24
Downloaded image 20/24
Downloaded image 21/24
Downloaded image 22/24
Downloaded image 23/24
Downloaded image 24/24
Saving GeoTIFF. Please wait...
Image saved to data/images/Allison Elem/tif/Allison Elem Building 0.tif
Downloaded image 1/3564
Downloaded image 2/3564
Downloaded image 3/3564
Downloaded image 4/3564
Downloaded image 5/3564
Downloaded image 6/3564
Downloaded image 7/3564
Downloaded image 8/3564
Downloaded image 9/3564
Downloaded image 10/3564
Downloaded image 11/3564
Downloaded image 12/3564
Downloaded image 13/3564
Downloaded image 14/3564
Downloaded im

0it [00:00, ?it/s]

Downloaded image 1/306
Downloaded image 2/306
Downloaded image 3/306
Downloaded image 4/306
Downloaded image 5/306
Downloaded image 6/306
Downloaded image 7/306
Downloaded image 8/306
Downloaded image 9/306
Downloaded image 10/306
Downloaded image 11/306
Downloaded image 12/306
Downloaded image 13/306
Downloaded image 14/306
Downloaded image 15/306
Downloaded image 16/306
Downloaded image 17/306
Downloaded image 18/306
Downloaded image 19/306
Downloaded image 20/306
Downloaded image 21/306
Downloaded image 22/306
Downloaded image 23/306
Downloaded image 24/306
Downloaded image 25/306
Downloaded image 26/306
Downloaded image 27/306
Downloaded image 28/306
Downloaded image 29/306
Downloaded image 30/306
Downloaded image 31/306
Downloaded image 32/306
Downloaded image 33/306
Downloaded image 34/306
Downloaded image 35/306
Downloaded image 36/306
Downloaded image 37/306
Downloaded image 38/306
Downloaded image 39/306
Downloaded image 40/306
Downloaded image 41/306
Downloaded image 42/306
D

0it [00:00, ?it/s]

Downloaded image 1/144
Downloaded image 2/144
Downloaded image 3/144
Downloaded image 4/144
Downloaded image 5/144
Downloaded image 6/144
Downloaded image 7/144
Downloaded image 8/144
Downloaded image 9/144
Downloaded image 10/144
Downloaded image 11/144
Downloaded image 12/144
Downloaded image 13/144
Downloaded image 14/144
Downloaded image 15/144
Downloaded image 16/144
Downloaded image 17/144
Downloaded image 18/144
Downloaded image 19/144
Downloaded image 20/144
Downloaded image 21/144
Downloaded image 22/144
Downloaded image 23/144
Downloaded image 24/144
Downloaded image 25/144
Downloaded image 26/144
Downloaded image 27/144
Downloaded image 28/144
Downloaded image 29/144
Downloaded image 30/144
Downloaded image 31/144
Downloaded image 32/144
Downloaded image 33/144
Downloaded image 34/144
Downloaded image 35/144
Downloaded image 36/144
Downloaded image 37/144
Downloaded image 38/144
Downloaded image 39/144
Downloaded image 40/144
Downloaded image 41/144
Downloaded image 42/144
D

Convert each building TIF to png for use in Roboflow inference

In [3]:
from PIL import Image



  0%|          | 0/15 [00:00<?, ?it/s]